<a href="https://colab.research.google.com/github/tam8738/KHDL/blob/main/cleanData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/KHDL/data/enron_spam_data.csv')  # Thay đường dẫn tương ứng
print(data.head())  # Hiển thị 5 dòng đầu tiên của dữ liệu

   Message ID                       Subject  \
0           0  christmas tree farm pictures   
1           1      vastar resources , inc .   
2           2  calpine daily gas nomination   
3           3                    re : issue   
4           4     meter 7268 nov allocation   

                                             Message Spam/Ham        Date  
0                                                NaN      ham  1999-12-10  
1  gary , production from the high island larger ...      ham  1999-12-13  
2             - calpine daily gas nomination 1 . doc      ham  1999-12-14  
3  fyi - see note below - already done .\nstella\...      ham  1999-12-14  
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  1999-12-14  


In [104]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack

In [105]:
# Kiểm tra giá trị bị thiếu
print(data.isnull().sum())

Message ID      0
Subject       289
Message       371
Spam/Ham        0
Date            0
dtype: int64


In [106]:
#thay thế các dữ liệu bị thiếu trong cột message và subject
data['Message'].fillna('No Message', inplace=True)
data['Subject'].fillna('No Subject', inplace=True)

# Xóa các cột không cần thiết
data = data.drop(columns=['Message ID', 'Date'])

# Kiểm tra lại dữ liệu sau khi xử lý
print(data.isnull().sum())
print(data.head())

Subject     0
Message     0
Spam/Ham    0
dtype: int64
                        Subject  \
0  christmas tree farm pictures   
1      vastar resources , inc .   
2  calpine daily gas nomination   
3                    re : issue   
4     meter 7268 nov allocation   

                                             Message Spam/Ham  
0                                         No Message      ham  
1  gary , production from the high island larger ...      ham  
2             - calpine daily gas nomination 1 . doc      ham  
3  fyi - see note below - already done .\nstella\...      ham  
4  fyi .\n- - - - - - - - - - - - - - - - - - - -...      ham  


<ipython-input-106-74bc9127cf8d>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Message'].fillna('No Message', inplace=True)
<ipython-input-106-74bc9127cf8d>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [107]:
# Kiểm tra số lượng dòng trùng lặp
print(data.duplicated().sum())

3222


In [108]:
# Loại bỏ các dòng trùng lặp
data = data.drop_duplicates()

In [109]:
print(data.isnull().sum())

Subject     0
Message     0
Spam/Ham    0
dtype: int64


In [110]:
#Chuyển nhãn Spam/Ham thành giá trị số
data['Spam/Ham'] = data['Spam/Ham'].map({'ham': 0, 'spam': 1})

In [112]:
# 2. Kết hợp cột 'Subject' và 'Message' thành một cột văn bản duy nhất
data['Text'] = data['Subject'] + " " + data['Message']

In [114]:
from sklearn.utils import resample

# Tách dữ liệu thành hai nhóm
ham = data[data['Spam/Ham'] == 0]
spam = data[data['Spam/Ham'] == 1]

# Giảm số lượng mẫu của ham để cân bằng với spam
ham_downsampled = resample(ham,
                           replace=False,  # Không lấy lại mẫu
                           n_samples=len(spam),  # Số lượng mẫu bằng với spam
                           random_state=42)

# Kết hợp lại thành DataFrame cân bằng
balanced_data = pd.concat([ham_downsampled, spam])

# Kiểm tra phân phối nhãn sau khi cân bằng
print("Phân phối nhãn sau khi undersampling:")
print(balanced_data['Spam/Ham'].value_counts())

Phân phối nhãn sau khi undersampling:
Spam/Ham
0    14584
1    14584
Name: count, dtype: int64


In [115]:
# 3. Áp dụng TF-IDF để chuyển đổi văn bản thành dạng số
tfidf_vectorizer = TfidfVectorizer()  # Khởi tạo vectorizer
X_text = tfidf_vectorizer.fit_transform(data['Text'])  # Vector hóa dữ liệu văn bản

In [116]:
# 4. Chuẩn hóa đặc trưng số (Độ dài tin nhắn)
data['Message Length'] = data['Message'].apply(len)  # Tính độ dài tin nhắn
scaler = MinMaxScaler()  # Khởi tạo Min-Max Scaler
data['Message Length Normalized'] = scaler.fit_transform(data[['Message Length']])  # Chuẩn hóa

# Tạo một scaler để chuẩn hóa giá trị
scaler = MinMaxScaler()

# Chuẩn hóa cột `Message Length` và thêm vào DataFrame
data['Message Length Normalized'] = scaler.fit_transform(data[['Message Length']])

# Kiểm tra kết quả
print(data[['Message Length', 'Message Length Normalized']].head())

   Message Length  Message Length Normalized
0              10                   0.000039
1            4282                   0.018747
2              38                   0.000162
3            1171                   0.005124
4            1124                   0.004918


In [ ]:
# 5. Kết hợp dữ liệu văn bản và đặc trưng đã chuẩn hóa
# Chuyển 'Message Length Normalized' thành dạng ma trận để kết hợp
X_features = data[['Message Length Normalized']].values
X_combined = hstack([X_text, X_features])  # Kết hợp văn bản và đặc trưng số

In [118]:
data.to_csv("/content/drive/MyDrive/KHDL/data/processed_enron1.csv", index=False)

print("Dữ liệu đã xử lý được lưu vào file 'processed_enron_spam_data.csv'.")

Dữ liệu đã xử lý được lưu vào file 'processed_enron_spam_data.csv'.
